In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
credits = pd.read_csv('E://Pratheeksha//recommender system//tmdb_5000_credits.csv')
movies = pd.read_csv("E://Pratheeksha//recommender system//tmdb_5000_movies.csv")

#**Movies dataset**

Columns such as 'genres', 'keywords', 'production_companies', 'production_countries', and 'spoken_languages' are in json format which needs to be cleaned for further analysis

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies.shape

(4803, 20)

In [5]:
movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [7]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [8]:
movies = movies[movies.astype(str)['genres'] != '[]']

In [9]:
movies['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
5       [{"id": 14, "name": "Fantasy"}, {"id": 28, "na...
6       [{"id": 16, "name": "Animation"}, {"id": 10751...
7       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
8       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
9       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
10      [{"id": 12, "name": "Adventure"}, {"id": 14, "...
11      [{"id": 12, "name": "Adventure"}, {"id": 28, "...
12      [{"id": 12, "name": "Adventure"}, {"id": 14, "...
13      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
14      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
15      [{"id": 12, "name": "Adventure"}, {"id": 10751...
16      [{"id": 878, "name": "Science Fiction"}, {"id"...
17      [{"id"

**Converting the json format to a list of strings**


In [10]:
import json
import warnings
from ast import literal_eval
warnings.filterwarnings('ignore')
from IPython.display import HTML

In [11]:
df=movies

In [12]:
# Parse stringed list features into python objects
features = ['keywords', 'genres','production_companies', 'production_countries', 'spoken_languages']
for i in features:
    df[i] = movies[i].apply(literal_eval)
    

In [13]:
def list_genres(x):
    l = [d['name'] for d in x]
    return(l)
df['genres'] = df['genres'].apply(list_genres)

In [14]:
def list_keywords(x):
    l = [d['name'] for d in x]
    return(l)
df['keywords'] = df['keywords'].apply(list_keywords)

In [15]:
def list_companies(x):
    l = [d['name'] for d in x]
    return(l)
df['production_companies'] = df['production_companies'].apply(list_companies)

In [16]:
def list_countries(x):
    l = [d['name'] for d in x]
    return(l)
df['production_countries'] = df['production_countries'].apply(list_countries)

In [17]:
def list_languages(x):
    l = [d['name'] for d in x]
    return(l)
df['spoken_languages'] = df['spoken_languages'].apply(list_languages)

In [18]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


#**Credits Dataset**

In [19]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [20]:
credits.shape

(4803, 4)

In [21]:
credits.isnull().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [22]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
movie_id    4803 non-null int64
title       4803 non-null object
cast        4803 non-null object
crew        4803 non-null object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [23]:
df1 = credits.copy()

In [24]:
features = ['cast','crew']
for i in features:
    df1[i] = credits[i].apply(literal_eval)

Extracting only the director from crew since other crew members dont matter much

In [25]:
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
df1['crew']=df1['crew'].apply(director)
df1.rename(columns={'crew':'director'},inplace=True)

Converting json format to list of strings for cast column

In [26]:
def list_cast(x):
    l = [d['name'] for d in x]
    return(l)
df1['cast'] = df1['cast'].apply(list_cast)

In [27]:
df1.head()

,movie_id,title,cast,director
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


In [28]:
df1 = df1[df1.astype(str)['cast'] != '[]']

In [29]:
df1.rename(columns = {'movie_id':'id'}, inplace = True) 

In [30]:
df1.head()

,id,title,cast,director
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


#**Extracting Features**


Merging both datasets

In [31]:
alldata = df.merge(df1, on = 'id')

Checking if title_x from movies dataset and title_y from credits dataset match

In [32]:
alldata['title_y'].equals(alldata['title_x'])

True

Checking if title_x from movies dataset and original_title from credits dataset match

In [33]:
alldata['original_title'].equals(alldata['title_x'])

False

Extracting only the year from release date

In [34]:
alldata['year'] = pd.DatetimeIndex(alldata['release_date']).year

Dropping the unnessary columns along with title_x and original_title(because it contains title in another language)

In [35]:
alldata.drop(columns=['status', 'original_title', 'homepage', 'id', 'spoken_languages', 'original_title', 'title_x','release_date' ], axis=1, inplace=True)

In [36]:
alldata.head(2)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,revenue,runtime,tagline,vote_average,vote_count,title_y,cast,director,year
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2787965087,162.0,Enter the World of Pandora.,7.2,11800,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,2009
1,300000000,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,2007


Renaming title_y to title

In [37]:
alldata.rename(columns = {'title_y':'title'}, inplace = True) 

Combining vote_average and vote_count

In [38]:
#Calculating mean of vote average
c=alldata['vote_average'].mean()
print(c)

6.123841617523163


In [39]:
#calculating the vote for 60 percentile movie 
m=alldata['vote_count'].quantile(0.6)
m

377.0

In [40]:
filtered_movies=alldata.copy().loc[alldata['vote_count']>=m]
filtered_movies.shape

(1901, 17)

Calculating the score

In [41]:
def imdbscore(x,m=m,c=c):
    v=x['vote_count']
    r=x['vote_average']
    #remember the imdb formula 
    return (v/(v+m) * r) + (m/(m+v) * c)

In [42]:
alldata['score']=filtered_movies.apply(imdbscore,axis=1)

In [43]:
alldata['score'] = round(alldata['score'], 3)
alldata.head(2)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,revenue,runtime,tagline,vote_average,vote_count,title,cast,director,year,score
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2787965087,162.0,Enter the World of Pandora.,7.2,11800,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,2009,7.167
1,300000000,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,2007,6.840


Extracting only top 3 members of the cast since they would be the leads

In [44]:
def top5(x):
    if len(x) > 3:
        x = x[:3]
    return(x)
alldata['cast'] = alldata['cast'].apply(top5)

In [45]:
alldata.head(2)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,revenue,runtime,tagline,vote_average,vote_count,title,cast,director,year,score
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2787965087,162.0,Enter the World of Pandora.,7.2,11800,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,2009,7.167
1,300000000,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,2007,6.840


Creating a copy of alldata and making some more changes, since we might need alldata at a later point

In [46]:
new_df=alldata.copy()

Strip Spaces and Convert to Lowercase from cast.

Mention Director 2 times to give it more weight relative to the entire cast.

In [47]:
new_df['cast'] = new_df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
new_df['dir'] = new_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
new_df['dir'] = new_df['dir'].apply(lambda x: [x,x, x])

Removing empty lists from cast and keywords

In [48]:
new_df = new_df[new_df.astype(str)['cast'] != '[]']
new_df = new_df[new_df.astype(str)['keywords'] != '[]']

Calculating the frequenct counts of every keyword that appears in the dataset.

In [49]:
s = new_df.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

independent film        317
woman director          317
duringcreditsstinger    307
based on novel          197
murder                  189
Name: keyword, dtype: int64

Limiting the frequent counts above 1. We do not have any use for keywords that occur only once.

In [50]:
s = s[s > 1]
s

independent film         317
woman director           317
duringcreditsstinger     307
based on novel           197
murder                   189
aftercreditsstinger      170
violence                 150
dystopia                 139
sport                    126
revenge                  118
sex                      111
friendship               106
biography                105
musical                  105
teenager                  99
3d                        98
love                      95
sequel                    94
suspense                  92
new york                  89
police                    84
los angeles               81
high school               80
alien                     76
prison                    72
nudity                    71
family                    69
superhero                 69
drug                      69
london england            69
                        ... 
journal                    2
witchcraft                 2
play                       2
supervillain  

In [51]:
#taking the indices from the dataframe
indices = pd.Series(new_df.index, index=new_df['title'])

Function to consider the keywords that occur in s(frequent keywords)

In [52]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

Stemmers remove morphological affixes from words, leaving only the word stem.
for eg dogs-> dog, running-> run

Stripping and converting to lowercase

In [53]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
new_df['keywords'] = new_df['keywords'].apply(filter_keywords)
new_df['keywords'] = new_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
new_df['keywords'] = new_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [54]:
new_df['keywords']

0       [cultureclash, futur, spacewar, spacecoloni, s...
1       [ocean, drugabus, exoticisland, eastindiatradi...
2       [spi, basedonnovel, secretag, sequel, mi6, bri...
3       [dccomic, crimefight, terrorist, secretident, ...
4       [basedonnovel, mar, medallion, spacetravel, pr...
5       [dualident, amnesia, sandstorm, loveofone'slif...
6       [hostag, magic, hors, fairytal, music, princes...
7       [marvelcom, sequel, superhero, basedoncomicboo...
8       [witch, magic, broom, schoolofwitchcraft, wiza...
9       [dccomic, vigilant, superhero, basedoncomicboo...
10      [savingtheworld, dccomic, invulner, sequel, su...
11        [kill, undercov, secretag, britishsecretservic]
12      [witch, fortunetel, bondag, exoticisland, mons...
13      [texa, hors, survivor, texasrang, partner, out...
14      [savingtheworld, dccomic, superhero, basedonco...
15      [basedonnovel, fictionalplac, brothersisterrel...
16      [newyork, shield, marvelcom, superhero, basedo...
17      [sea, 

Merging keywords cast director and genres into one column and joining them with a space


In [55]:
new_df['allwords'] = new_df['keywords'] + new_df['cast'] + new_df['dir'] + new_df['genres']
new_df['allwords'] = new_df['allwords'].apply(lambda x: ' '.join(x))

Convert a collection of text to a matrix of token counts



In [56]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(new_df['allwords'])

Finding the similarity using Cosine Similarity technique

In [57]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [58]:
new_df = new_df.reset_index()
titles = new_df['title']
indices = pd.Series(new_df.index, index=new_df['title'])

#Recommending Movies


Function to recommend movies based on similarity and popularity

In [59]:
def recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    qualified = new_df.iloc[movie_indices][['title', 'year']]
    mov = new_df.iloc[movie_indices][['vote_count', 'vote_average']]
    #qualified = mov[(mov['vote_count'] >= m) & (mov['vote_count'].notnull()) & (mov['vote_average'].notnull())]
    #qualified['vote_count'] = qualified['vote_count'].astype('int')
    #qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified.rename(columns={'title':'Title'},inplace=True)
    qualified.rename(columns={'year':'Year'},inplace=True)
    qualified['Director']=new_df['director']
    qualified['IMDBScore'] = mov.apply(imdbscore, axis=1)
    qualified = qualified.sort_values('IMDBScore', ascending=False).head(10)
    qualified['IMDBScore']=round(qualified['IMDBScore'], 2)
    #qualified['cast']=alldata['cast']
    
    return qualified

Making recommendation using an example

In [60]:
recommendations('The Dark Knight Rises')

,Title,Year,Director,IMDBScore
65,The Dark Knight,2008,Christopher Nolan,8.14
94,Inception,2010,Christopher Nolan,8.05
93,Interstellar,2014,Christopher Nolan,8.03
3367,Memento,2000,Christopher Nolan,7.93
1179,The Prestige,2006,Christopher Nolan,7.85
117,Batman Begins,2005,Christopher Nolan,7.43
1020,Insomnia,2002,Christopher Nolan,6.63
2353,Running Scared,2006,Wayne Kramer,6.53
424,Batman Returns,1992,Tim Burton,6.51
2790,Trash,2014,Stephen Daldry,6.51


In [61]:
recommendations('Inception')

,Title,Year,Director,IMDBScore
65,The Dark Knight,2008,Christopher Nolan,8.14
93,Interstellar,2014,Christopher Nolan,8.03
3367,Memento,2000,Christopher Nolan,7.93
1179,The Prestige,2006,Christopher Nolan,7.85
3,The Dark Knight Rises,2012,Christopher Nolan,7.54
117,Batman Begins,2005,Christopher Nolan,7.43
273,Minority Report,2002,Steven Spielberg,6.98
1020,Insomnia,2002,Christopher Nolan,6.63
1537,Looper,2012,Rian Johnson,6.56
898,Super 8,2011,J.J. Abrams,6.54
